In [4]:
!ls DATD

DATD+Rand_training.csv	DATD_and_DATD+Rand_test.csv  DATD_training.csv


In [5]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 24.1 MB 2.0 MB/s            
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 1.3 MB/s            
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [15]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ccmilne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ccmilne/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
random_train_data = pd.read_csv("datd/DATD+Rand_training.csv")
train_data = pd.read_csv("datd/DATD_training.csv")

In [10]:
train_data['text'].head(3)

0               @user Me talking to my antidepressants
1    going to attempt at least to change this sour ...
2            flighty sag placements make me anxioussss
Name: text, dtype: object

In [11]:
train_data['label'].value_counts()

MENTAL_HEALTH    473
OTHER            427
Name: label, dtype: int64

In [12]:
def preprocessing(tweets_array):
    
    """
    Take in an array of tweets, and return the processed tweets.
    
    (e.g. input: 'i am depressed', output - 'am depressed')  -> since I remove those words with length 1
    
    """
    
    processed_array = []
    
    for tweet in tqdm(tweets_array):
        
        # remove other non-alphabets symbols with space (i.e. keep only alphabets and whitespaces).
        processed = re.sub('[^@a-zA-Z ]', '', tweet)

        #Usernames (anything that begins with "@")
        no_usernames = " ".join([x for x in processed.split() if x[0] != '@'])
        
        #words = processed.split()
        words = word_tokenize(no_usernames)

        #Stopwords
        tokens_without_sw = [word for word in words if not word in stopwords.words()]
        
        # keep words that have length of more than 1 (e.g. gb, bb), remove those with length 1.
        processed_array.append(' '.join([word for word in tokens_without_sw if len(word) > 1]))
    
    return processed_array

In [13]:
train_data['processed'] = preprocessing(train_data['text'])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [01:25<00:00, 10.51it/s]


In [14]:
train_data

,text,label,processed
0,@user Me talking to my antidepressants,MENTAL_HEALTH,Me talking antidepressants
1,going to attempt at least to change this sour ...,MENTAL_HEALTH,going attempt least change sour mood Its mainl...
2,flighty sag placements make me anxioussss,OTHER,flighty sag placements make anxioussss
3,I'm going to buy cream cheese after work and s...,MENTAL_HEALTH,Im going buy cream cheese work bathtub eat Thi...
4,My anxiety and stress levels for today are act...,MENTAL_HEALTH,My anxiety stress levels today actually making...
...,...,...,...
895,Libra culture is using the same nail polish co...,OTHER,Libra culture using nail polish color salon vi...
896,"anyways, my last year of college starts today ...",OTHER,anyways last year college starts today whew an...
897,"'Dad.. I'm depressed.' 'Hi depressed, I'm Dad.'",OTHER,Dad Im depressed Hi depressed Im Dad
898,respiratory depression be my god,OTHER,respiratory depression god


In [16]:
x = np.asarray(train_data['processed'])
y = np.asarray(train_data['label'])

label_map = {cat: index for index, cat in enumerate(np.unique(y))}
y_prep = np.asarray([label_map[l] for l in y])

label_map

{'MENTAL_HEALTH': 0, 'OTHER': 1}

## Training Word Embeddings

In [17]:
x_tokenized = [[w for w in sentence.split(" ") if w != ""] for sentence in x]
x_tokenized[0]

['Me', 'talking', 'antidepressants']

In [21]:
import time
from gensim.models import Word2Vec

start = time.time()

model = Word2Vec(
            x_tokenized,
            vector_size=900 # Size is the length of our vector.
        )

end = round(time.time()-start, 2)
print("This process took",end,"seconds.")

This process took 0.07 seconds.


In [22]:
model.wv.most_similar("depression")

[('anxiety', 0.6839107871055603),
 ('depressed', 0.6795772910118103),
 ('Im', 0.6524881720542908),
 ('feel', 0.6221291422843933),
 ('dont', 0.6006389856338501),
 ('anxious', 0.5870542526245117),
 ('know', 0.5678699016571045),
 ('people', 0.5413738489151001),
 ('get', 0.5381070375442505),
 ('feeling', 0.5354692935943604)]

In [23]:
class Sequencer():
    
    def __init__(self,
                 all_words,
                 max_words,
                 seq_len,
                 embedding_matrix
                ):
        
        self.seq_len = seq_len
        self.embed_matrix = embedding_matrix
        """
        temp_vocab = Vocab which has all the unique words
        self.vocab = Our last vocab which has only most used N words.
    
        """
        temp_vocab = list(set(all_words))
        self.vocab = []
        self.word_cnts = {}
        """
        Now we'll create a hash map (dict) which includes words and their occurencies
        """
        for word in temp_vocab:
            # 0 does not have a meaning, you can add the word to the list
            # or something different.
            count = len([0 for w in all_words if w == word])
            self.word_cnts[word] = count
            counts = list(self.word_cnts.values())
            indexes = list(range(len(counts)))
        
        # Now we'll sort counts and while sorting them also will sort indexes.
        # We'll use those indexes to find most used N word.
        cnt = 0
        while cnt + 1 != len(counts):
            cnt = 0
            for i in range(len(counts)-1):
                if counts[i] < counts[i+1]:
                    counts[i+1],counts[i] = counts[i],counts[i+1]
                    indexes[i],indexes[i+1] = indexes[i+1],indexes[i]
                else:
                    cnt += 1
        
        for ind in indexes[:max_words]:
            self.vocab.append(temp_vocab[ind])
                    
    def textToVector(self,text):
        # First we need to split the text into its tokens and learn the length
        # If length is shorter than the max len we'll add some spaces (100D vectors which has only zero values)
        # If it's longer than the max len we'll trim from the end.
        tokens = text.split()
        len_v = len(tokens)-1 if len(tokens) < self.seq_len else self.seq_len-1
        vec = []
        for tok in tokens[:len_v]:
            try:
                vec.append(self.embed_matrix[tok])
            except Exception as E:
                pass
        
        last_pieces = self.seq_len - len(vec)
        for i in range(last_pieces):
            vec.append(np.zeros(100,))
        
        return np.asarray(vec).flatten()

In [35]:
all_words = [token for seq in x_tokenized for token in seq]
max_words = len(all_words)

In [36]:
sequencer = Sequencer(
                all_words = all_words,
                max_words = max_words,
                seq_len = 15,
                embedding_matrix = model.wv
            )

In [40]:
test_vec = sequencer.textToVector("i am in love with you")
test_vec[0][:10]

/tmp/ipykernel_172/2991566974.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(vec).flatten()


array([ 0.00011371,  0.00107021,  0.0002885 ,  0.00085611,  0.00125629,
       -0.00129817,  0.00017513,  0.00118605,  0.00035596,  0.00015801],
      dtype=float32)

In [41]:
test_vec.shape

(15,)

## Modelling

In [42]:
x_vecs = np.asarray([sequencer.textToVector(" ".join(seq)) for seq in x_tokenized])
print(x_vecs.shape)

(900,)


/tmp/ipykernel_172/2991566974.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(vec).flatten()
/tmp/ipykernel_172/524339332.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_vecs = np.asarray([sequencer.textToVector(" ".join(seq)) for seq in x_tokenized])


In [43]:
X_train, X_test, y_train, y_test = train_test_split(x_vecs, y_prep, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(720,)
(180,)
(720,)
(180,)


In [44]:
start = time.time() 

svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))
svm_classifier.score(X_test, y_test)

ValueError: setting an array element with a sequence.